In [19]:
#https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
#https://www.kaggle.com/hassanamin/atis-airlinetravelinformationsystem
# !pip install bert-for-tf2
# !pip install sentencepiece

In [44]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import pandas as pd
import re
import random
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

In [2]:
data=pd.read_csv("/home/carlton/Downloads/atis_intents_train.csv", header=0,names=['intent','text'])

In [3]:
data['text'][0]

' what flights are available from pittsburgh to baltimore on thursday morning'

In [6]:
data['intent'].value_counts()

atis_flight            3665
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: intent, dtype: int64

In [137]:
data['intent'][5]

'atis_aircraft'

In [138]:
data['text'][5]

' what kind of aircraft is used on a flight from cleveland to dallas'

In [57]:
thisdict = {
  "atis_flight": 0,
  "atis_airfare": 1,
  "atis_ground_service":2,
  "atis_airline":3,
  "atis_abbreviation":4,  
  "atis_aircraft":5,
  "atis_flight_time":6,
  "atis_flight_time":7,
  "atis_quantity":8
}

In [61]:
y=data['intent'].map(thisdict)

In [7]:
data.isnull().values.any()

False

In [8]:
data.shape

(4833, 2)

In [9]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [10]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [11]:
questions = []
sentences = list(data['text'])
for sen in sentences:
    questions.append(preprocess_text(sen.strip()))

In [12]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [13]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [14]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [15]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [16]:
tokenized_reviews = [tokenize_reviews(question) for question in questions]

In [17]:
tokenized_reviews

[[2054, 7599, 2024, 2800, 2013, 6278, 2000, 6222, 2006, 9432, 2851],
 [2054,
  2003,
  1996,
  5508,
  2051,
  1999,
  2624,
  3799,
  2005,
  1996,
  2572,
  3462,
  2975,
  2899],
 [10036, 4355, 2250, 17883, 2013, 22954, 2000, 10108],
 [2461, 4440, 27092, 2013, 6278, 2000, 4407, 2104, 6363],
 [1045, 2342, 3462, 4826, 2013, 8912, 2000, 11334],
 [2054, 2785, 1997, 2948, 2003, 2109, 2006, 3462, 2013, 6044, 2000, 5759],
 [2265, 2033, 1996, 7599, 2013, 6278, 2000, 3050, 3349, 2006, 9432],
 [2035, 7599, 2013, 3731, 2000, 2899],
 [2054, 2785, 1997, 2598, 5193, 2003, 2800, 1999, 7573],
 [2265, 2033, 1996, 7599, 2013, 5759, 2000, 2624, 3799],
 [2265,
  2033,
  1996,
  7599,
  2013,
  2624,
  5277,
  2000,
  12948,
  2011,
  2126,
  1997,
  5395],
 [2054, 2003, 1996, 10036, 4355, 3462, 2013, 3731, 2000, 1038, 9148],
 [2035, 7599, 2000, 6222, 2044, 7610],
 [2265, 2033, 1996, 2034, 2465, 27092, 2013, 3731, 2000, 7573],
 [2265, 2033, 1996, 2598, 5193, 1999, 7573],
 [2035, 7599, 2013, 7573, 2000, 

In [22]:
tokenized_reviews

[[2054, 7599, 2024, 2800, 2013, 6278, 2000, 6222, 2006, 9432, 2851],
 [2054,
  2003,
  1996,
  5508,
  2051,
  1999,
  2624,
  3799,
  2005,
  1996,
  2572,
  3462,
  2975,
  2899],
 [10036, 4355, 2250, 17883, 2013, 22954, 2000, 10108],
 [2461, 4440, 27092, 2013, 6278, 2000, 4407, 2104, 6363],
 [1045, 2342, 3462, 4826, 2013, 8912, 2000, 11334],
 [2054, 2785, 1997, 2948, 2003, 2109, 2006, 3462, 2013, 6044, 2000, 5759],
 [2265, 2033, 1996, 7599, 2013, 6278, 2000, 3050, 3349, 2006, 9432],
 [2035, 7599, 2013, 3731, 2000, 2899],
 [2054, 2785, 1997, 2598, 5193, 2003, 2800, 1999, 7573],
 [2265, 2033, 1996, 7599, 2013, 5759, 2000, 2624, 3799],
 [2265,
  2033,
  1996,
  7599,
  2013,
  2624,
  5277,
  2000,
  12948,
  2011,
  2126,
  1997,
  5395],
 [2054, 2003, 1996, 10036, 4355, 3462, 2013, 3731, 2000, 1038, 9148],
 [2035, 7599, 2000, 6222, 2044, 7610],
 [2265, 2033, 1996, 2034, 2465, 27092, 2013, 3731, 2000, 7573],
 [2265, 2033, 1996, 2598, 5193, 1999, 7573],
 [2035, 7599, 2013, 7573, 2000, 

In [62]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [63]:
reviews_with_len

[[[2054, 7599, 2024, 2800, 2013, 6278, 2000, 6222, 2006, 9432, 2851], 0, 11],
 [[2054,
   2003,
   1996,
   5508,
   2051,
   1999,
   2624,
   3799,
   2005,
   1996,
   2572,
   3462,
   2975,
   2899],
  7,
  14],
 [[10036, 4355, 2250, 17883, 2013, 22954, 2000, 10108], 1, 8],
 [[2461, 4440, 27092, 2013, 6278, 2000, 4407, 2104, 6363], 1, 9],
 [[1045, 2342, 3462, 4826, 2013, 8912, 2000, 11334], 0, 8],
 [[2054, 2785, 1997, 2948, 2003, 2109, 2006, 3462, 2013, 6044, 2000, 5759],
  5,
  12],
 [[2265, 2033, 1996, 7599, 2013, 6278, 2000, 3050, 3349, 2006, 9432], 0, 11],
 [[2035, 7599, 2013, 3731, 2000, 2899], 0, 6],
 [[2054, 2785, 1997, 2598, 5193, 2003, 2800, 1999, 7573], 2, 9],
 [[2265, 2033, 1996, 7599, 2013, 5759, 2000, 2624, 3799], 0, 9],
 [[2265,
   2033,
   1996,
   7599,
   2013,
   2624,
   5277,
   2000,
   12948,
   2011,
   2126,
   1997,
   5395],
  0,
  13],
 [[2054, 2003, 1996, 10036, 4355, 3462, 2013, 3731, 2000, 1038, 9148], 0, 11],
 [[2035, 7599, 2000, 6222, 2044, 7610], 0

In [64]:

random.shuffle(reviews_with_len)

In [65]:
reviews_with_len.sort(key=lambda x: x[2])

In [66]:
reviews_with_len

[[[2149], 0, 1],
 [[2028, 2126], 0, 2],
 [[3642, 21461], 4, 2],
 [[2598, 5193, 2899], 2, 3],
 [[6278, 2000, 7573], 0, 3],
 [[12948, 2000, 6044], 0, 3],
 [[5759, 2598, 5193], 2, 3],
 [[2598, 3665, 7573], 2, 3],
 [[2054, 2003, 6522], 4, 3],
 [[2598, 5193, 7573], 2, 3],
 [[2598, 5193, 9182], 2, 3],
 [[6222, 2000, 4407], 0, 3],
 [[12948, 2000, 6044], 0, 3],
 [[2598, 2624, 3799], 2, 3],
 [[9182, 2000, 4407], 0, 3],
 [[2598, 5193, 6222], 2, 3],
 [[2054, 2003, 2004], 4, 3],
 [[3190, 2000, 9184], 0, 3],
 [[7599, 2013, 5548], 0, 3],
 [[2598, 5193, 5865], 2, 3],
 [[9182, 2000, 7573], 0, 3],
 [[2003, 2149, 2250], 3, 3],
 [[2054, 2003, 7842], 4, 3],
 [[2054, 2003, 21469], 4, 3],
 [[4863, 16840, 9706], 4, 3],
 [[9375, 8582, 6522], 3, 3],
 [[6278, 2000, 7573], 0, 3],
 [[2054, 2515, 2812], 4, 3],
 [[9375, 8582, 2149], 4, 3],
 [[5865, 2598, 5193], 2, 3],
 [[2598, 3665, 6708], 2, 3],
 [[2054, 2003, 25423], 4, 3],
 [[2054, 2003, 7842], 4, 3],
 [[7573, 2000, 5865], 0, 3],
 [[6278, 2000, 7573], 0, 3],
 [[

In [67]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [84]:
sorted_reviews_labels

[([2149], 0),
 ([2028, 2126], 0),
 ([3642, 21461], 4),
 ([2598, 5193, 2899], 2),
 ([6278, 2000, 7573], 0),
 ([12948, 2000, 6044], 0),
 ([5759, 2598, 5193], 2),
 ([2598, 3665, 7573], 2),
 ([2054, 2003, 6522], 4),
 ([2598, 5193, 7573], 2),
 ([2598, 5193, 9182], 2),
 ([6222, 2000, 4407], 0),
 ([12948, 2000, 6044], 0),
 ([2598, 2624, 3799], 2),
 ([9182, 2000, 4407], 0),
 ([2598, 5193, 6222], 2),
 ([2054, 2003, 2004], 4),
 ([3190, 2000, 9184], 0),
 ([7599, 2013, 5548], 0),
 ([2598, 5193, 5865], 2),
 ([9182, 2000, 7573], 0),
 ([2003, 2149, 2250], 3),
 ([2054, 2003, 7842], 4),
 ([2054, 2003, 21469], 4),
 ([4863, 16840, 9706], 4),
 ([9375, 8582, 6522], 3),
 ([6278, 2000, 7573], 0),
 ([2054, 2515, 2812], 4),
 ([9375, 8582, 2149], 4),
 ([5865, 2598, 5193], 2),
 ([2598, 3665, 6708], 2),
 ([2054, 2003, 25423], 4),
 ([2054, 2003, 7842], 4),
 ([7573, 2000, 5865], 0),
 ([6278, 2000, 7573], 0),
 ([9375, 8582, 25423], 4),
 ([2054, 2003, 8764], 4),
 ([6222, 2000, 4407], 0),
 ([3731, 2000, 6278], 0),
 ([

In [68]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))


In [83]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [69]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [70]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [71]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [94]:
batched_dataset

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [72]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [78]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES =9 

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [79]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [80]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [81]:
train_data

<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [82]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
137/137 [==============================] - 7s 49ms/step - loss: 0.5156 - sparse_categorical_accuracy: 0.8599
Epoch 2/5
137/137 [==============================] - 7s 50ms/step - loss: 0.1069 - sparse_categorical_accuracy: 0.9676
Epoch 3/5
137/137 [==============================] - 7s 49ms/step - loss: 0.0316 - sparse_categorical_accuracy: 0.9917
Epoch 4/5
137/137 [==============================] - 7s 48ms/step - loss: 0.0118 - sparse_categorical_accuracy: 0.9968
Epoch 5/5
137/137 [==============================] - 6s 47ms/step - loss: 0.0049 - sparse_categorical_accuracy: 0.9993


In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))


In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [139]:
test=["what kind of aircraft is used on a flight from cleveland to dallas"]

In [140]:
tokenized_reviews = [tokenize_reviews(question) for question in test]

In [141]:
tokenized_reviews

[[2054,
  2785,
  1997,
  2948,
  2003,
  2109,
  2006,
  1037,
  3462,
  2013,
  6044,
  2000,
  5759]]

In [142]:
reviews_with_len = [[review, 1 , len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [143]:
reviews_with_len

[[[2054,
   2785,
   1997,
   2948,
   2003,
   2109,
   2006,
   1037,
   3462,
   2013,
   6044,
   2000,
   5759],
  1,
  13]]

In [144]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [145]:
sorted_reviews_labels

[([2054,
   2785,
   1997,
   2948,
   2003,
   2109,
   2006,
   1037,
   3462,
   2013,
   6044,
   2000,
   5759],
  1)]

In [146]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [147]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [150]:
BATCH_SIZE = 1
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [151]:
batched_dataset

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [ ]:
np.argmax(text_model.predict(batched_dataset))